In [6]:
import sys, random, math, pickle
import numpy as np
import torch
from sklearn.utils import shuffle
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
sys.path.append('../DG/gan')
from sklearn.model_selection import train_test_split

In [7]:
# def count_parameters(model):
#     return sum(p.numel() for p in model.parameters() if p.requires_grad)
# count_parameters(model)

In [8]:
# In real data, if flag sum is 1 --> Then no timestep at all. --> So we do remove those ones by converting them to zeros, then remove from the list
# In real data, there is no flag of length ZERO
def remove_zero_datapoints(X,Y,mask):
    indices_non_zero = torch.nonzero(torch.sum(mask,1)-1).squeeze()
    return X[indices_non_zero], Y[indices_non_zero], mask[indices_non_zero]

def get_one_class(X,Y,Y_label,flag,class_label): # (X, Y, and mask) are the whole dataset that is consisted of many classes, Y is NOT One-Hot Encoded
    indices_class_label = np.where(Y_label==class_label)
    return X[indices_class_label], Y[indices_class_label], Y_label[indices_class_label],flag[indices_class_label]

def get_n_samples(X,Y,flag,n_samples):
    randomList = random.sample(range(0, Y.shape[0]), n_samples)
    return X[randomList], Y[randomList], flag[randomList]

## Import -> Remove Zeros -> Split classes

In [9]:
training_real = np.load('../data/google/data_train.npz')

real_train_X = torch.from_numpy(training_real['data_feature']).float() #[50000, 2500, 9]
real_train_Y = torch.from_numpy(training_real['data_attribute']) #[50000,4]
real_train_flags = torch.from_numpy(training_real['data_gen_flag'])   # (50000, 2500)

real_train_X,real_train_Y,real_train_flags = remove_zero_datapoints(real_train_X,real_train_Y,real_train_flags)
real_train_Y_labels = torch.argmax(real_train_Y,1) #[50000,]  


real_train_X0, real_train_Y0, real_train_Y_lables0, real_train_flag0 = get_one_class(real_train_X, real_train_Y,
                                                                                    real_train_Y_labels,real_train_flags,0)
real_train_X1, real_train_Y1, real_train_Y_labels1, real_train_flag1 = get_one_class(real_train_X, real_train_Y,
                                                                                    real_train_Y_labels,real_train_flags,1)
real_train_X2, real_train_Y2, real_train_Y_labels2, real_train_flag2 = get_one_class(real_train_X, real_train_Y,
                                                                                    real_train_Y_labels,real_train_flags,2)
real_train_X3, real_train_Y3, real_train_Y_labels3, real_train_flag3 = get_one_class(real_train_X, real_train_Y,
                                                                                    real_train_Y_labels,real_train_flags,3)


In [5]:
# -- We will divide the train dataset into 50% train, and 0% validation. 

In [18]:
n_class0 = real_train_X0.shape[0]
n_class1 = real_train_X1.shape[0]
n_class2 = real_train_X2.shape[0]
n_class3 = real_train_X3.shape[0]

In [19]:
n_class0

6250

In [7]:
# real_train_X0,real_train_Y0,real_train_flag0 = get_n_samples(real_train_X0, real_train_Y0,real_train_flag0,
#                                                              n_samples=int(n_class0*0.7))

# real_train_X1,real_train_Y1,real_train_flag1 = get_n_samples(real_train_X1, real_train_Y1,real_train_flag1,
#                                                             n_samples=int(n_class1*0.7))

# real_train_X2,real_train_Y2,real_train_flag2 = get_n_samples(real_train_X2, real_train_Y2,real_train_flag2,
#                                                             n_samples=int(n_class2*0.7))

# real_train_X3,real_train_Y3,real_train_flag3 = get_n_samples(real_train_X3, real_train_Y3,real_train_flag3,
#                                                             n_samples=int(n_class3*0.7))

In [20]:
real_train_X_reduced= torch.cat((real_train_X0[:int(n_class0*0.5)],real_train_X1[:int(n_class1*0.5)],
                                 real_train_X2[:int(n_class2*0.5)],real_train_X3[:int(n_class3*0.5)]))

real_train_Y_reduced= torch.cat((real_train_Y0[:int(n_class0*0.5)],real_train_Y1[:int(n_class1*0.5)],
                                 real_train_Y2[:int(n_class2*0.5)],real_train_Y3[:int(n_class3*0.5)]))

real_train_flags_reduced = torch.cat((real_train_flag0[:int(n_class0*0.5)],real_train_flag1[:int(n_class1*0.5)],
                                      real_train_flag2[:int(n_class2*0.5)],real_train_flag3[:int(n_class3*0.5)]))

In [23]:
real_train_X_reduced, real_train_Y_reduced, real_train_flags_reduced = shuffle(real_train_X_reduced, 
                                                                               real_train_Y_reduced, real_train_flags_reduced )
np.savez('../data/google/data_train_reduced',data_feature=real_train_X_reduced,data_attribute= real_train_Y_reduced, 
         data_gen_flag=real_train_flags_reduced)

In [24]:
real_val_X= torch.cat((real_train_X0[int(n_class0*0.5):],real_train_X1[int(n_class1*0.5):],
                       real_train_X2[int(n_class2*0.5):],real_train_X3[int(n_class3*0.5):]))

real_val_Y= torch.cat((real_train_Y0[int(n_class0*0.5):],real_train_Y1[int(n_class1*0.5):],
                       real_train_Y2[int(n_class2*0.5):],real_train_Y3[int(n_class3*0.5):]))

real_val_flags = torch.cat((real_train_flag0[int(n_class0*0.5):],real_train_flag1[int(n_class1*0.5):],
                            real_train_flag2[int(n_class2*0.5):],real_train_flag3[int(n_class3*0.5):]))

In [25]:
real_val_X, real_val_Y, real_train_flags_reduced = shuffle(real_val_X ,real_val_Y, real_val_flags )
np.savez('../data/google/data_train_val',data_feature=real_val_X,data_attribute= real_val_Y, 
         data_gen_flag=real_val_flags)

# Test Data

In [7]:
#Note: I am not going to chanage the variables names, it's the same train", I just changed the np.load and np.savez
training_real = np.load('data/google/data_test.npz')

real_train_X = torch.from_numpy(training_real['data_feature']).float() #[50000, 2500, 9]
real_train_Y = torch.from_numpy(training_real['data_attribute']) #[50000,4]
real_train_flags = torch.from_numpy(training_real['data_gen_flag'])   # (50000, 2500)

real_train_X,real_train_Y,real_train_flags = remove_zero_datapoints(real_train_X,real_train_Y,real_train_flags)
real_train_Y_labels = torch.argmax(real_train_Y,1) #[50000,]  


real_train_X0, real_train_Y0, real_train_Y_lables0, real_train_flag0 = get_one_class(real_train_X, real_train_Y,
                                                                                    real_train_Y_labels,real_train_flags,0)
real_train_X1, real_train_Y1, real_train_Y_labels1, real_train_flag1 = get_one_class(real_train_X, real_train_Y,
                                                                                    real_train_Y_labels,real_train_flags,1)
real_train_X2, real_train_Y2, real_train_Y_labels2, real_train_flag2 = get_one_class(real_train_X, real_train_Y,
                                                                                    real_train_Y_labels,real_train_flags,2)
real_train_X3, real_train_Y3, real_train_Y_labels3, real_train_flag3 = get_one_class(real_train_X, real_train_Y,
                                                                                    real_train_Y_labels,real_train_flags,3)


## n samples from each class

In [8]:
real_train_X0,real_train_Y0,real_train_flag0 = get_n_samples(real_train_X0, real_train_Y0,real_train_flag0,
                                                             n_samples=real_train_X0.shape[0]//2)

real_train_X1,real_train_Y1,real_train_flag1 = get_n_samples(real_train_X1, real_train_Y1,real_train_flag1,
                                                            n_samples=real_train_X1.shape[0]//2)

real_train_X2,real_train_Y2,real_train_flag2 = get_n_samples(real_train_X2, real_train_Y2,real_train_flag2,
                                                            n_samples=real_train_X2.shape[0]//2)

real_train_X3,real_train_Y3,real_train_flag3 = get_n_samples(real_train_X3, real_train_Y3,real_train_flag3,
                                                            n_samples=real_train_X3.shape[0]//2)

In [9]:
real_train_X_reduced= torch.cat((real_train_X0,real_train_X1,real_train_X2,real_train_X3))
real_train_Y_reduced= torch.cat((real_train_Y0,real_train_Y1,real_train_Y2,real_train_Y3))
real_train_flags_reduced = torch.cat((real_train_flag0,real_train_flag1,real_train_flag2,real_train_flag3))

In [10]:
real_train_X_reduced, real_train_Y_reduced, real_train_flags_reduced = shuffle(real_train_X_reduced, 
                                                                               real_train_Y_reduced, real_train_flags_reduced )

In [11]:
np.savez('data/google/data_test_reduced',data_feature=real_train_X_reduced,data_attribute= real_train_Y_reduced, 
         data_gen_flag=real_train_flags_reduced)